### Adjust Word Level

In [ ]:
#!pip install xlsxwriter

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

#### While Loop

In [ ]:
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language
lang_pair = "Italian"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [ ]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/3-Adjust Word Level/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)  # create path

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
#df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

In [ ]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

In [ ]:
df_pair_all = pd.read_excel(f"{lang_pair_list[0]}")
df_pair_all

In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif"]

In [ ]:
words = df_pair_all["dict_entry_main"].values.tolist()

In [ ]:
disable_set = set(disable_list)
words_set = set(words)

In [ ]:
df_pair = pd.DataFrame(list(words_set.difference(disable_set)), columns=["dict_entry_main"])
df_pair

In [ ]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [ ]:
def word_count_result(df,column_list): # df is dataframe, bigger_than is integer
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list)
    df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [ ]:
def word_count_bool(df, word_thresh_num, column_list): # df is a dataframe, bigger_than is an integer
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list)
    df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"count"}, inplace=True)
    
    return  df_word_count["count"][df_word_count.loc[:,"count"] > word_thresh_num].any()  

#### Condition 1

In [ ]:
# while loop code block word and twogram pair
twogram_num = 100  # 2*step_num minimum: for each word takes two twogram
twogram_pair_num = 100  # 2*step_num minimum: for each word takes two twogram pair
word_thresh_num = 2

word_start = 0  # 0
word_end = 7  # 10
step_num = word_end  # 10
word_limit = 28  # 200
part_num = 1
while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    ngram_list = []
    for i in df_pair["dict_entry_main"]:
        for j in word_tokenize(i):
            for k in df_word["word"]:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)

    # output
    df_output_result = pd.concat([df_word, df_lang_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
    a = 0

    for i in range(0,110):
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word 
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert twogram pair
        try:
            var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
            freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
            if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, word_thresh_num, [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                        a += 1
                        var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
                        freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

    df_lesson_word_count = word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

#### Condition 2

In [ ]:
part_result_file = glob.glob(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result*.xlsx")
part_result_file

In [ ]:
# Read previous part result
df_part_all = pd.DataFrame()
for i in part_result_file:
    df_var = pd.read_excel(f"{i}")
    df_part_all = pd.concat([df_part_all,df_var], axis=0)
df_part_twogram_pair = df_part_all.loc[:,[f"twogram_pair_{lang_pair.lower()}"]]
df_part_twogram_pair.reset_index(drop=True, inplace=True)
set_part_twogram_pair = set(df_part_twogram_pair[f"twogram_pair_{lang_pair.lower()}"])

# while loop code block
twogram_num = 100  # 2*step_num minimum: for each word takes two twogram
twogram_pair_num = 100  # 2*step_num minimum: for each word takes two twogram pair
word_thresh_num = 7

word_start = 0  # 0
word_end = 28  # 10
step_num = word_end  # 10
word_limit = 28  # 200
part_num = 1
while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    ngram_list = []
    for i in df_pair["dict_entry_main"]:
        for j in word_tokenize(i):
            for k in df_word["word"]:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)
    set_lang_pair_twogram = set(df_lang_pair_twogram[f"twogram_pair_{lang_pair.lower()}"])
    df_set_result = pd.DataFrame(set_lang_pair_twogram.difference(set_part_twogram_pair))
    df_set_result.rename(columns={0:f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_set_pair_twogram = pd.merge(df_lang_pair_twogram, df_set_result, how="inner", on=f"twogram_pair_{lang_pair.lower()}")

    # twogram
    word_list = df_word["word"].values.tolist()
    data_kind = "twogram"
    twogram_list  = df_twogram_sent.iloc[:,0].values.tolist()
    
    resultlist2 = []

    manager = multiprocessing.Manager()
    resultlist2 = manager.list()
    
    def word_in_wordgroup2(list_var2):
        mergelist = []
        try:
            word = list_var2.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist2.append(list_var2)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup2, twogram_list) # string_word liste 

    result_list2 = list(resultlist2)
    df_result2 = pd.DataFrame(result_list2)
    df_result2 = df_result2.rename(columns = {0: f"{data_kind}"})
    df_result2.iloc[:,0] = df_result2.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
    df_merge2 = pd.merge(df_result2, df_twogram_sent, how="inner", on=f"{data_kind}")
    df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
    df_merge_result2.drop_duplicates(inplace=True)
    df_merge_result2.reset_index(drop=True, inplace=True)
    df_twogram_result = df_merge_result2
    #df_twogram_result = df_twogram_result.head(100)

    # output
    df_output_result = pd.concat([df_word, df_twogram_result, df_set_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word","twogram","freq_twogram",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
    a = 0
    b = 0

    for i in range(0,110):
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert n grams
        try:
            var1 = df_output_result.iloc[a,2]
            freq_var1 = df_output_result.iloc[a,3]
            if (len(df_lesson_result["twogram"]) < twogram_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,"twogram"] = var1
                df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num-5), ["twogram"]): # word count result                
                        a += 1
                        var1 = df_output_result.iloc[a,2]
                        freq_var1 = df_output_result.iloc[a,3]
                        df_lesson_result.loc[i,"twogram"] = var1
                        df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,"twogram"] = np.nan
                    df_lesson_result.loc[i,"freq_twogram"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

        try:
            var2 = df_output_result.iloc[b,4]
            freq_var2 = df_output_result.iloc[b,5]
            if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num-5), [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                        b += 1
                        var2 = df_output_result.iloc[b,4]
                        freq_var2 = df_output_result.iloc[b,5]
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
            else:
                pass
        except:
            pass
        b += 1

    df_lesson_word_count = word_count_result(df_lesson_result, ["twogram",f"twogram_pair_{lang_pair.lower()}"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

#### Check Result

In [ ]:
df_lesson_result

In [ ]:
#word_count_result(df_lesson_result, ["twogram"])

In [ ]:
#word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])

#### Read Output File And Multi Write

In [ ]:
df_part_11 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result1.xlsx")
df_part_11

In [ ]:
df_part_12 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result2.xlsx")
df_part_12

In [ ]:
df_part_13 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result3.xlsx")
df_part_13

In [ ]:
df_part_14 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result4.xlsx")
df_part_14

In [ ]:
df_part_21 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_28_Result1.xlsx")
df_part_21

In [ ]:
writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Result_With_Frequency.xlsx", engine='xlsxwriter')

In [ ]:
df_part_11.to_excel(writer, sheet_name='Word_Part11', index=False)
df_part_12.to_excel(writer, sheet_name='Word_Part12', index=False)
df_part_13.to_excel(writer, sheet_name='Word_Part13', index=False)
df_part_14.to_excel(writer, sheet_name='Word_Part14', index=False)
df_part_21.to_excel(writer, sheet_name='Word_Part21', index=False)

In [ ]:
writer.save()

#### Output File Word Order

In [ ]:
df_part_11

In [ ]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [ ]:
word_in_twogram(df_part_11, "word", f"twogram_pair_{lang_pair.lower()}") 

In [ ]:
df_word_order_11 = word_in_twogram(df_part_11, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_12 = word_in_twogram(df_part_12, "word", f"twogram_pair_{lang_pair.lower()}") 
df_word_order_13 = word_in_twogram(df_part_13, "word", f"twogram_pair_{lang_pair.lower()}") 
df_word_order_14 = word_in_twogram(df_part_14, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_21 = word_in_twogram(df_part_21, "word", f"twogram") 
df_word_order_212 = word_in_twogram(df_part_21, "word", f"twogram_pair_{lang_pair.lower()}")  

In [ ]:
df_word_order_join_11 = df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_12 = df_word_order_12.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_13 = df_word_order_13.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_14 = df_word_order_14.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_211 = df_word_order_21.groupby(["word"])["twogram"].apply(", ".join).reset_index()
df_word_order_join_212 = df_word_order_212.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_21 = pd.merge(df_word_order_join_211, df_word_order_join_212, how="outer", on="word")

In [ ]:
df_word_order_join_21

In [ ]:
writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Join_Result_Without_Frequency.xlsx", engine='xlsxwriter')

In [ ]:
df_word_order_join_11.to_excel(writer2, sheet_name='Word_Part11', index=False)
df_word_order_join_12.to_excel(writer2, sheet_name='Word_Part12', index=False)
df_word_order_join_13.to_excel(writer2, sheet_name='Word_Part13', index=False)
df_word_order_join_14.to_excel(writer2, sheet_name='Word_Part14', index=False)
df_word_order_join_21.to_excel(writer2, sheet_name='Word_Part21', index=False)

In [ ]:
writer2.save()

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/3-Adjust Word Level/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass